The aim of this notebook is to generate the initial training set (or initial plate) for the subsequent active learning loop.

# Python libraries imports

In [2]:
import numpy as np
import random

# Create a folder to save the resulting dataset

In [3]:
data_folder = "echo_files"

**Important**  
The seed was not set when the file was first executed.  
The concentrations used for the initial plate in the paper can be found in **data/plate_AL_1_raw_everything.csv**.

**Caution**  
When you run the code under the pre-existing files architecture, the results are overwritten in the **initial_plate.csv** dataset.

# Defining the tested concentrations and volumes

The concentrations below are defined as a proportion of the maximum allowed concentration.

In [4]:
# Allowed concentration ratios for each component. Mostly, four ratios were chosen for each component.

mg_glutamate_ratios = [0.1, 0.3, 0.5, 1]
k_glutamate_ratios = [0.1, 0.3, 0.5, 1]
pga_ratios = [0.1, 0.3, 0.5, 1]
nad_ratios = [0.1, 0.3, 0.5, 1]
folinic_acid_ratios = [0.1, 0.3, 0.5, 1]
spermidine_ratios = [0.1, 0.3, 0.5, 1]
aa_ratios = [0.1, 0.3, 0.5, 1]
trna_ratios = [0.1, 0.3, 0.5, 1]
sodiumoxalate_ratios = [0.1, 0.3, 0.5, 1]
nucleotides_mix_ratios = [0.1, 0.3, 0.5, 1]
dna_mix_ratios = [0, 1]

hepes_ratios = [1]
extract_ratios = [1]
promoter_ratios = [1]
rbs_ratios = [1]

# hmp_ratios = []
# glucose_ratios = []
# pep_ratios = []
# glycogen_ratios = []
# k2hpo4_ratios = []
# glucose6p_ratios = []
# phosphocreatine_ratios = []
# creatinekinase_ratios = []
# maltodextrin_ratios = []
# gsh_ratios = []
# gssg_ratios = []
# peg_ratios = []
# promoter_ratios = []
# RBS_ratios = []
# coa_ratios = []
# camp_ratios = []
# peg_ratios = []

In [5]:
# Maximum concentration for each component. Resulting concentration = Maximum concentration * Ratio.

mg_glutamate_max = 10.4
k_glutamate_max = 355
sodiumoxalate_max = 2.1 
pga_max = 30
nucleotides_mix_max = 1.5 
nad_max = 30
folinic_acid_max = 0.2
spermidine_max = 1.3
aa_max = 2
trna_max = 0.5

dna_mix_max = 10
hepes_max = 50
extract_max = 30

# pic_max = 0
# promoter_max = 10
# RBS_max = 10
# hmp_max = 0.8
# glucose_max = 30
# pep_max = 30
# glycogen_max = 32
# k2hpo4_max = 10
# glucose6p_max = 33
# phosphocreatine_max = 67
# creatinekinase_max = 250
# maltodextrin_max = 35
# gsh_max = 1
# gssg_max = 0.1
# pep_max = 30

# Defining control samples (maximum and autofluorescence)

This defines the control samples (reference) used in the first plate, that have maximum concentrations for all components. 
The autofluorescence **<sup>1</sup>** control has no DNA. **<sup>1</sup>** is the fluorescence measured in the cell-free reaction supplemented with water and using the reference composition.

In [14]:
# Reference extract where all components are at the maximum concentrations.

maximum_extract = np.ones((1, 13), dtype = 'int')

max_sample_conc = [maximum_extract[0, 0] * mg_glutamate_max, 
                   maximum_extract[0, 1] * k_glutamate_max, 
                   maximum_extract[0, 2] * sodiumoxalate_max,
                   maximum_extract[0, 3] * pga_max,
                   maximum_extract[0, 4] * nucleotides_mix_max,
                   maximum_extract[0, 5] * nad_max,
                   maximum_extract[0, 6] * folinic_acid_max,
                   maximum_extract[0, 7] * spermidine_max,
                   maximum_extract[0, 8] * aa_max,
                   maximum_extract[0, 9] * trna_max,
                   maximum_extract[0, 10] * dna_mix_max,
                   maximum_extract[0, 11] * hepes_max,
                   maximum_extract[0, 12] * extract_max
                  ]

max_sample_conc

[10.4, 355, 2.1, 30, 1.5, 30, 0.2, 1.3, 2, 0.5, 10, 50, 30]

In [9]:
# Reference autofluorescence extract 

autofluorescence_extract = np.ones((1, 13), dtype = 'int')

autofluorescence_sample_conc = [autofluorescence_extract[0, 0] * mg_glutamate_max, 
                                autofluorescence_extract[0, 1] * k_glutamate_max, 
                                autofluorescence_extract[0, 2] * sodiumoxalate_max,
                                autofluorescence_extract[0, 3] * pga_max,
                                autofluorescence_extract[0, 4] * nucleotides_mix_max,
                                autofluorescence_extract[0, 5] * nad_max,
                                autofluorescence_extract[0, 6] * folinic_acid_max,
                                autofluorescence_extract[0, 7] * spermidine_max,
                                autofluorescence_extract[0, 8] * aa_max,
                                autofluorescence_extract[0, 9] * trna_max,
                                autofluorescence_extract[0, 10] * 0, # No DNA in the autofluorescence control
                                autofluorescence_extract[0, 11] * hepes_max,
                                autofluorescence_extract[0, 12] * extract_max
                               ]

autofluorescence_sample_conc

[10.4, 355, 2.1, 30, 1.5, 30, 0.2, 1.3, 2, 0.5, 0, 50, 30]

In [16]:
max_sample_conc = np.reshape(max_sample_conc, (1, 13))

autofluorescence_sample_conc = np.asarray(autofluorescence_sample_conc)
autofluorescence_sample_conc = np.reshape(autofluorescence_sample_conc, (1, 13))

ctrl_array = max_sample_conc
ctrl_array = np.concatenate((ctrl_array, autofluorescence_sample_conc), axis = 0)

ctrl_array

array([[1.04e+01, 3.55e+02, 2.10e+00, 3.00e+01, 1.50e+00, 3.00e+01,
        2.00e-01, 1.30e+00, 2.00e+00, 5.00e-01, 1.00e+01, 5.00e+01,
        3.00e+01],
       [1.04e+01, 3.55e+02, 2.10e+00, 3.00e+01, 1.50e+00, 3.00e+01,
        2.00e-01, 1.30e+00, 2.00e+00, 5.00e-01, 0.00e+00, 5.00e+01,
        3.00e+01]])

# All components at maximum, except one at minimum 

This defines a set of combinations for the initial plate as follows:  
* 10 forced combinations: All the components are at the maximum concentration, except one at the minimum.
* 1  forced comibnation: All the components are at the maximum concentration.

In [11]:
all_max_except_one_low = max_sample_conc

In [12]:
mg_glutamate = mg_glutamate_ratios[-1]
k_glutamate = k_glutamate_ratios[-1]
sodiumoxalate = sodiumoxalate_ratios[-1]
pga = pga_ratios[-1]
nucleotides_mix = nucleotides_mix_ratios[-1]
nad = nad_ratios[-1]
folinic_acid = folinic_acid_ratios[-1]
spermidine = spermidine_ratios[-1]
aa = aa_ratios[-1]
trna = trna_ratios[-1]

dna = dna_mix_ratios[1]
hepes = hepes_ratios[0]
extract = extract_ratios[0]

# peg = peg_ratios[0]
# promoter = promoter_ratios[0]
# RBS = RBS_ratios[0]
# coa = coa_ratios[-1]
# camp = camp_ratios[-1]

In [13]:
this_sample_conc = [0.1 * mg_glutamate_max,
                    k_glutamate * k_glutamate_max,
                    sodiumoxalate * sodiumoxalate_max,
                    pga * pga_max,
                    nucleotides_mix * nucleotides_mix_max,
                    nad * nad_max,
                    folinic_acid * folinic_acid_max,
                    spermidine * spermidine_max,
                    aa * aa_max,
                    trna * trna_max,
                    dna * dna_mix_max,
                    hepes * hepes_max,
                    extract * extract_max]

this_sample_conc = np.reshape(this_sample_conc, (1, 13)) 
all_max_except_one_low = np.concatenate((all_max_except_one_low, this_sample_conc), axis = 0)

array([[1.04e+00, 3.55e+02, 2.10e+00, 3.00e+01, 1.50e+00, 3.00e+01,
        2.00e-01, 1.30e+00, 2.00e+00, 5.00e-01, 1.00e+01, 5.00e+01,
        3.00e+01]])

In [134]:
this_sample_conc = [mg_glutamate * mg_glutamate_max,
                    0.1 * k_glutamate_max,
                    sodiumoxalate * sodiumoxalate_max,
                    pga * pga_max,
                    nucleotides_mix * nucleotides_mix_max,
                    nad * nad_max,
                    folinic_acid * folinic_acid_max,
                    spermidine * spermidine_max,
                    aa * aa_max,
                    trna * trna_max,
                    dna * dna_mix_max,
                    hepes * hepes_max,
                    extract * extract_max]

this_sample_conc = np.reshape(this_sample_conc, (1, 13)) 
all_max_except_one_low = np.concatenate((all_max_except_one_low, this_sample_conc), axis = 0)

In [135]:
this_sample_conc = [mg_glutamate * mg_glutamate_max,
                    k_glutamate * k_glutamate_max,
                    0.1 * sodiumoxalate_max,
                    pga * pga_max,
                    nucleotides_mix * nucleotides_mix_max,
                    nad * nad_max,
                    folinic_acid * folinic_acid_max,
                    spermidine * spermidine_max,
                    aa * aa_max,
                    trna * trna_max,
                    dna * dna_mix_max,
                    hepes * hepes_max,
                    extract * extract_max]

this_sample_conc = np.reshape(this_sample_conc, (1, 13)) 
all_max_except_one_low = np.concatenate((all_max_except_one_low, this_sample_conc), axis = 0)

In [136]:
this_sample_conc = [mg_glutamate * mg_glutamate_max,
                    k_glutamate * k_glutamate_max,
                    sodiumoxalate * sodiumoxalate_max,
                    0.1 * pga_max,
                    nucleotides_mix * nucleotides_mix_max,
                    nad * nad_max,
                    folinic_acid * folinic_acid_max,
                    spermidine * spermidine_max,
                    aa * aa_max,
                    trna * trna_max,
                    dna * dna_mix_max,
                    hepes * hepes_max,
                    extract * extract_max]

this_sample_conc = np.reshape(this_sample_conc, (1, 13)) 
all_max_except_one_low = np.concatenate((all_max_except_one_low, this_sample_conc), axis = 0)

In [137]:
this_sample_conc = [mg_glutamate * mg_glutamate_max,
                    k_glutamate * k_glutamate_max,
                    sodiumoxalate * sodiumoxalate_max,
                    pga * pga_max,
                    0.1 * nucleotides_mix_max,
                    nad * nad_max,
                    folinic_acid * folinic_acid_max,
                    spermidine * spermidine_max,
                    aa * aa_max,
                    trna * trna_max,
                    dna * dna_mix_max,
                    hepes * hepes_max,
                    extract * extract_max]

this_sample_conc = np.reshape(this_sample_conc, (1, 13)) 
all_max_except_one_low = np.concatenate((all_max_except_one_low, this_sample_conc), axis = 0)

In [138]:
this_sample_conc = [mg_glutamate * mg_glutamate_max,
                    k_glutamate * k_glutamate_max,
                    sodiumoxalate * sodiumoxalate_max,
                    pga * pga_max,
                    nucleotides_mix * nucleotides_mix_max,
                    0.1 * nad_max,
                    folinic_acid * folinic_acid_max,
                    spermidine * spermidine_max,
                    aa * aa_max,
                    trna * trna_max,
                    dna * dna_mix_max,
                    hepes * hepes_max,
                    extract * extract_max]

this_sample_conc = np.reshape(this_sample_conc, (1, 13)) 
all_max_except_one_low = np.concatenate((all_max_except_one_low, this_sample_conc), axis = 0)

In [139]:
this_sample_conc = [mg_glutamate * mg_glutamate_max,
                    k_glutamate * k_glutamate_max,
                    sodiumoxalate * sodiumoxalate_max,
                    pga * pga_max,
                    nucleotides_mix * nucleotides_mix_max,
                    nad * nad_max,
                    0.1 * folinic_acid_max,
                    spermidine * spermidine_max,
                    aa * aa_max,
                    trna * trna_max,
                    dna * dna_mix_max,
                    hepes * hepes_max,
                    extract * extract_max]

this_sample_conc = np.reshape(this_sample_conc, (1, 13)) 
all_max_except_one_low = np.concatenate((all_max_except_one_low, this_sample_conc), axis = 0)

In [140]:
this_sample_conc = [mg_glutamate * mg_glutamate_max,
                    k_glutamate * k_glutamate_max,
                    sodiumoxalate * sodiumoxalate_max,
                    pga * pga_max,
                    nucleotides_mix * nucleotides_mix_max,
                    nad * nad_max,
                    folinic_acid * folinic_acid_max,
                    0.1 * spermidine_max,
                    aa * aa_max,
                    trna * trna_max,
                    dna * dna_mix_max,
                    hepes * hepes_max,
                    extract * extract_max]

this_sample_conc = np.reshape(this_sample_conc, (1, 13)) 
all_max_except_one_low = np.concatenate((all_max_except_one_low, this_sample_conc), axis = 0)

In [141]:
this_sample_conc = [mg_glutamate * mg_glutamate_max,
                    k_glutamate * k_glutamate_max,
                    sodiumoxalate * sodiumoxalate_max,
                    pga * pga_max,
                    nucleotides_mix * nucleotides_mix_max,
                    nad * nad_max,
                    folinic_acid * folinic_acid_max,
                    spermidine * spermidine_max,
                    0.1 * aa_max,
                    trna * trna_max,
                    dna * dna_mix_max,
                    hepes * hepes_max,
                    extract * extract_max]

this_sample_conc = np.reshape(this_sample_conc, (1, 13)) 
all_max_except_one_low = np.concatenate((all_max_except_one_low, this_sample_conc), axis = 0)

In [142]:
this_sample_conc = [mg_glutamate * mg_glutamate_max,
                    k_glutamate * k_glutamate_max,
                    sodiumoxalate * sodiumoxalate_max,
                    pga * pga_max,
                    nucleotides_mix * nucleotides_mix_max,
                    nad * nad_max,
                    folinic_acid * folinic_acid_max,
                    spermidine * spermidine_max,
                    aa * aa_max,
                    0.1 * trna_max,
                    dna * dna_mix_max,
                    hepes * hepes_max,
                    extract * extract_max]

this_sample_conc = np.reshape(this_sample_conc, (1, 13)) 
all_max_except_one_low = np.concatenate((all_max_except_one_low, this_sample_conc), axis = 0)

In [143]:
this_sample_conc = [mg_glutamate * mg_glutamate_max,
                    k_glutamate * k_glutamate_max,
                    sodiumoxalate * sodiumoxalate_max,
                    pga * pga_max,
                    nucleotides_mix * nucleotides_mix_max,
                    nad * nad_max,
                    folinic_acid * folinic_acid_max,
                    spermidine * spermidine_max,
                    aa * aa_max,
                    trna * trna_max,
                    dna * dna_mix_max,
                    hepes * hepes_max,
                    extract * extract_max]

this_sample_conc = np.reshape(this_sample_conc, (1, 13)) 
all_max_except_one_low = np.concatenate((all_max_except_one_low, this_sample_conc), axis = 0)

In [144]:
print(all_max_except_one_low.shape)

(12, 13)


# All components at minimum, except one at maximum 

This defines a set of combinations for the initial plate as follows:  
* 10 forced combinations: All the components are at the minimum concentration, except one at the maximum.
* 1  forced comibnation: All the components are at the minimum concentration.

In [145]:
all_min_except_one_high = this_sample_conc

In [146]:
mg_glutamate = mg_glutamate_conc[0]
k_glutamate = k_glutamate_conc[0]
sodiumoxalate = sodiumoxalate_conc[0]
pga = pga_conc[0]
nucleotides_mix = nucleotides_mix_conc[0]
nad = nad_conc[0]
folinic_acid = folinic_acid_conc[0]
spermidine = spermidine_conc[0]
aa = aa_conc[0]
trna = trna_conc[0]

dna = dna_mix_conc[1]
hepes = hepes_conc[0]
extract = extract_conc[0]

In [147]:
this_sample_conc = [1 * mg_glutamate_max,
                    k_glutamate * k_glutamate_max,
                    sodiumoxalate * sodiumoxalate_max,
                    pga * pga_max,
                    nucleotides_mix * nucleotides_mix_max,
                    nad * nad_max,
                    folinic_acid * folinic_acid_max,
                    spermidine * spermidine_max,
                    aa * aa_max,
                    trna * trna_max,
                    dna * dna_mix_max,
                    hepes * hepes_max,
                    extract * extract_max]

this_sample_conc = np.reshape(this_sample_conc, (1, 13)) 
all_min_except_one_high = np.concatenate((all_min_except_one_high, this_sample_conc), axis = 0)

In [148]:
this_sample_conc = [mg_glutamate * mg_glutamate_max,
                    1 * k_glutamate_max,
                    sodiumoxalate * sodiumoxalate_max,
                    pga * pga_max,
                    nucleotides_mix * nucleotides_mix_max,
                    nad * nad_max,
                    folinic_acid * folinic_acid_max,
                    spermidine * spermidine_max,
                    aa * aa_max,
                    trna * trna_max,
                    dna * dna_mix_max,
                    hepes * hepes_max,
                    extract * extract_max]

this_sample_conc = np.reshape(this_sample_conc, (1, 13)) 
all_min_except_one_high = np.concatenate((all_min_except_one_high, this_sample_conc), axis = 0)

In [149]:
this_sample_conc = [mg_glutamate * mg_glutamate_max,
                    k_glutamate * k_glutamate_max,
                    1 * sodiumoxalate_max,
                    pga * pga_max,
                    nucleotides_mix * nucleotides_mix_max,
                    nad * nad_max,
                    folinic_acid * folinic_acid_max,
                    spermidine * spermidine_max,
                    aa * aa_max,
                    trna * trna_max,
                    dna * dna_mix_max,
                    hepes * hepes_max,
                    extract * extract_max]

this_sample_conc = np.reshape(this_sample_conc, (1, 13)) 
all_min_except_one_high = np.concatenate((all_min_except_one_high, this_sample_conc), axis = 0)

In [150]:
this_sample_conc = [mg_glutamate * mg_glutamate_max,
                    k_glutamate * k_glutamate_max,
                    sodiumoxalate * sodiumoxalate_max,
                    1 * pga_max,
                    nucleotides_mix * nucleotides_mix_max,
                    nad * nad_max,
                    folinic_acid * folinic_acid_max,
                    spermidine * spermidine_max,
                    aa * aa_max,
                    trna * trna_max,
                    dna * dna_mix_max,
                    hepes * hepes_max,
                    extract * extract_max]

this_sample_conc = np.reshape(this_sample_conc, (1, 13)) 
all_min_except_one_high = np.concatenate((all_min_except_one_high, this_sample_conc), axis = 0)

In [151]:
this_sample_conc = [mg_glutamate * mg_glutamate_max,
                    k_glutamate * k_glutamate_max,
                    sodiumoxalate * sodiumoxalate_max,
                    pga * pga_max,
                    1 * nucleotides_mix_max,
                    nad * nad_max,
                    folinic_acid * folinic_acid_max,
                    spermidine * spermidine_max,
                    aa * aa_max,
                    trna * trna_max,
                    dna * dna_mix_max,
                    hepes * hepes_max,
                    extract * extract_max]

this_sample_conc = np.reshape(this_sample_conc, (1, 13)) 
all_min_except_one_high = np.concatenate((all_min_except_one_high, this_sample_conc), axis = 0)

In [152]:
this_sample_conc = [mg_glutamate * mg_glutamate_max,
                    k_glutamate * k_glutamate_max,
                    sodiumoxalate * sodiumoxalate_max,
                    pga * pga_max,
                    nucleotides_mix * nucleotides_mix_max,
                    1 * nad_max,
                    folinic_acid * folinic_acid_max,
                    spermidine * spermidine_max,
                    aa * aa_max,
                    trna * trna_max,
                    dna * dna_mix_max,
                    hepes * hepes_max,
                    extract * extract_max]

this_sample_conc = np.reshape(this_sample_conc, (1, 13)) 
all_min_except_one_high = np.concatenate((all_min_except_one_high, this_sample_conc), axis = 0)

In [153]:
this_sample_conc = [mg_glutamate * mg_glutamate_max,
                    k_glutamate * k_glutamate_max,
                    sodiumoxalate * sodiumoxalate_max,
                    pga * pga_max,
                    nucleotides_mix * nucleotides_mix_max,
                    nad * nad_max,
                    1 * folinic_acid_max,
                    spermidine * spermidine_max,
                    aa * aa_max,
                    trna * trna_max,
                    dna * dna_mix_max,
                    hepes * hepes_max,
                    extract * extract_max]

this_sample_conc = np.reshape(this_sample_conc, (1, 13)) 
all_min_except_one_high = np.concatenate((all_min_except_one_high, this_sample_conc), axis = 0)

In [154]:
this_sample_conc = [mg_glutamate * mg_glutamate_max,
                    k_glutamate * k_glutamate_max,
                    sodiumoxalate * sodiumoxalate_max,
                    pga * pga_max,
                    nucleotides_mix * nucleotides_mix_max,
                    nad * nad_max,
                    folinic_acid * folinic_acid_max,
                    1 * spermidine_max,
                    aa * aa_max,
                    trna * trna_max,
                    dna * dna_mix_max,
                    hepes * hepes_max,
                    extract * extract_max]

this_sample_conc = np.reshape(this_sample_conc, (1, 13)) 
all_min_except_one_high = np.concatenate((all_min_except_one_high, this_sample_conc), axis = 0)

In [155]:
this_sample_conc = [mg_glutamate * mg_glutamate_max,
                    k_glutamate * k_glutamate_max,
                    sodiumoxalate * sodiumoxalate_max,
                    pga * pga_max,
                    nucleotides_mix * nucleotides_mix_max,
                    nad * nad_max,
                    folinic_acid * folinic_acid_max,
                    spermidine * spermidine_max,
                    1 * aa_max,
                    trna * trna_max,
                    dna * dna_mix_max,
                    hepes * hepes_max,
                    extract * extract_max]

this_sample_conc = np.reshape(this_sample_conc, (1, 13)) 
all_min_except_one_high = np.concatenate((all_min_except_one_high, this_sample_conc), axis = 0)

In [156]:
this_sample_conc = [mg_glutamate * mg_glutamate_max,
                    k_glutamate * k_glutamate_max,
                    sodiumoxalate * sodiumoxalate_max,
                    pga * pga_max,
                    nucleotides_mix * nucleotides_mix_max,
                    nad * nad_max,
                    folinic_acid * folinic_acid_max,
                    spermidine * spermidine_max,
                    aa * aa_max,
                    1 * trna_max,
                    dna * dna_mix_max,
                    hepes * hepes_max,
                    extract * extract_max]

this_sample_conc = np.reshape(this_sample_conc, (1, 13)) 
all_min_except_one_high = np.concatenate((all_min_except_one_high, this_sample_conc), axis = 0)

In [157]:
this_sample_conc = [mg_glutamate * mg_glutamate_max,
                    k_glutamate * k_glutamate_max,
                    sodiumoxalate * sodiumoxalate_max,
                    pga * pga_max,
                    nucleotides_mix * nucleotides_mix_max,
                    nad * nad_max,
                    folinic_acid * folinic_acid_max,
                    spermidine * spermidine_max,
                    aa * aa_max,
                    trna * trna_max,
                    dna * dna_mix_max,
                    hepes * hepes_max,
                    extract * extract_max]

this_sample_conc = np.reshape(this_sample_conc, (1, 13)) 
all_min_except_one_high = np.concatenate((all_min_except_one_high, this_sample_conc), axis = 0)

In [158]:
print(all_min_except_one_high.shape)

(12, 13)


# Filling the rest randomly

The rest of the plate will be filled randomly, while avoiding repeats.

In [159]:
answer_array = np.concatenate((ctrl_array, all_max_except_one_low, all_min_except_one_high), axis = 0)

In [160]:
print(answer_array.shape)

(26, 13)


In [161]:
def present_in_array_index(new_sample, array):
    
    """
    Verify if a sample is present in an array.
    """
    
    if array is None:
        return(False, -1)
    present = False
    new_sample = np.reshape(np.array(new_sample), (1, 13))
    for i in range(array.shape[0]):
        if np.array_equiv(array[i,:],new_sample):
            present = True
            break   
    return(present, i)

In [162]:
def generate_random_grid(array_to_avoid, sample_size = 100):
    active_learning_array = array_to_avoid
    if array_to_avoid is None:
        answerSize = 0
    else:
        answerSize = active_learning_array.shape[0]
    
    # Components at fixed concentrations
    extract = extract_ratios[0]
    dna = dna_mix_ratios[1]
    hepes = hepes_ratios[0]

    while answerSize < sample_size:
        this_sample = np.random.randint(0, high=3, size=11, dtype='int')
        this_sample_conc = [mg_glutamate_ratios[this_sample[0]] * mg_glutamate_max, 
                            k_glutamate_ratios[this_sample[1]] * k_glutamate_max,
                            sodiumoxalate_ratios[this_sample[2]] * sodiumoxalate_max,
                            pga_ratios[this_sample[3]] * pga_max,
                            nucleotides_mix_ratios[this_sample[4]] * nucleotides_mix_max,
                            nad_ratios[this_sample[5]] * nad_max,
                            folinic_acid_ratios[this_sample[6]] * folinic_acid_max,
                            spermidine_ratios[this_sample[7]] * spermidine_max,
                            aa_ratios[this_sample[8]] * aa_max,
                            trna_ratios[this_sample[9]] * trna_max, 
                            extract * extract_max,
                            dna * dna_mix_max,
                            hepes * hepes_max]
        
        this_sample_conc = np.reshape(this_sample_conc, (1, 13))
        if not present_in_array_index(this_sample_conc, active_learning_array)[0]:
            answerSize = answerSize + 1
            if active_learning_array is None:
                active_learning_array = this_sample_conc
            else:
                active_learning_array = np.concatenate((active_learning_array, this_sample_conc), axis = 0)
    return(active_learning_array)

In [163]:
answer_array = generate_random_grid(answer_array, sample_size = 102)

In [164]:
# Saving output in csv file
headers = "Mg-glutamate; k-glutamate; Sodium oxalate; 3-PGA; NTP; NAD; Folinic acid; Spermidine; tRNA; Amino acids; Extract; DNA; HEPES"
np.savetxt("{}/initial_plate_proCFPS.csv".format(data_folder), answer_array, delimiter=";", header = headers,fmt='%.5f')